## Imports

In [7]:
import pandas as pd
import os

## NDMA

In [ ]:
ndma_csv_df = pd.read_csv("../Scrapers/ndma_advisories_bulk.csv")
data_ndma = []

for _, row in ndma_csv_df.iterrows():
    source = row["source_agency"]
    posted_date = row["date"]
    title = row["title"]
    url = row["url"]
    filename, filetype = os.path.splitext(row["filename"])
    filetype = filetype.lstrip('.')

    data_ndma.append({
        "source": source,
        "posted_date": posted_date,
        "title": title,
        "url": url,
        "filename": filename,
        "filetype": filetype
    })

ndma_df = pd.DataFrame(data_ndma)

KeyError: 'source_agency'

In [9]:
ndma_df.head()

,source,posted_date,title,url,filename,filetype
0,NDMA,30-09-2025,"Well Marked Low-pressure Area over Saurashtra,...",https://www.ndma.gov.pk/storage/advisories/Sep...,EMaXMwnW0CqJnz5bB7uZ,.pdf
1,NDMA,29-09-2025,RAIN-WIND / THUNDERSHOWER PREDICTED IN SOUTH-E...,https://www.ndma.gov.pk/storage/advisories/Sep...,OWjJCJNscoxfI4L3MCzS,.pdf
2,NDMA,29-09-2025,Well Marked Low-Pressure Aera Over Gulf of Cam...,https://www.ndma.gov.pk/storage/advisories/Sep...,sWyC4mYYO9VZOop02Q2K,.pdf
3,NDMA,14-09-2025,NDMA's Weather Advisory dated 14 Sep 2025,https://www.ndma.gov.pk/storage/advisories/Sep...,ohUxnQnqhAGjinJQiGhs,.pdf
4,NDMA,07-09-2025,TORRENTIAL RAIN PREDICTED IN SINDH & OTHER PAR...,https://www.ndma.gov.pk/storage/advisories/Sep...,JyCxQN4qpEYEVCRYf1L9,.pdf
